In [1]:
using JuMP, CSV, DataFrames, Gurobi, Interpolations

In [2]:
# Read the test and train CSV files
data_targeted = CSV.File("/Users/sanyachauhan/Downloads/Data_Targeted.csv") |> DataFrame;
data_email = CSV.File("/Users/sanyachauhan/Downloads/Data_Emails.csv") |> DataFrame;
data_print = CSV.File("/Users/sanyachauhan/Downloads/Data_Print.csv") |> DataFrame;
data_influencer = CSV.File("/Users/sanyachauhan/Downloads/Data_Influencers.csv") |> DataFrame;

In [3]:
# Budget levels
B = [1000.0, 1500.0, 2000.0];

In [4]:
# Creating linear interpolation functions for each marketing option
interp_targeted = LinearInterpolation(data_targeted[!, :Cost], data_targeted[!, :Buys]);
interp_email = LinearInterpolation(data_email[!, :Cost], data_email[!, :Buys]);
interp_print = LinearInterpolation(data_print[!, :Cost], data_print[!, :Buys]);
interp_influencers = LinearInterpolation(data_influencer[!, :Cost], data_influencer[!, :Buys]);

In [5]:
for Budget in B
    # Optimization model
    model = Model(Gurobi.Optimizer)

    # views
    # Decision variables for investments
    @variable(model, investment_targeted >= 0)
    @variable(model, investment_email >= 0)
    @variable(model, investment_print >= 0)
    @variable(model, investment_influencers >= 0)
    
    # Example for Targeted Online Advertising
    @variable(model, targeted_segments[1:3] >= 0)
    breakpoints_targeted = [0, 40, 310, 990] # Example breakpoints
    slopes_targeted = [8, 10, 12] # Example slopes

    @constraint(model, sum(targeted_segments[i] for i in 1:3) <= breakpoints_targeted[end])
    for i in 1:3
        @constraint(model, targeted_segments[i] <= breakpoints_targeted[i+1] - breakpoints_targeted[i])
    end
    
    # Example for Email Marketing
    @variable(model, email_segments[1:3] >= 0)
    breakpoints_email = [0, 50, 200, 500] # Example breakpoints
    slopes_email = [2, 3, 4] # Example slopes
    @constraint(model, sum(email_segments[i] for i in 1:3) <= breakpoints_email[end])
    for i in 1:3
        @constraint(model, email_segments[i] <= breakpoints_email[i+1] - breakpoints_email[i])
    end
    
    # Example for Print Media
    @variable(model, print_segments[1:3] >= 0)
    breakpoints_print = [0, 100, 300, 700] # Example breakpoints
    slopes_print = [1, 2, 3] # Example slopes
    @constraint(model, sum(print_segments[i] for i in 1:3) <= breakpoints_print[end])
    for i in 1:3
        @constraint(model, print_segments[i] <= breakpoints_print[i+1] - breakpoints_print[i])
    end
    
    # Example for Influencer Marketing
    @variable(model, influencers_segments[1:3] >= 0)
    breakpoints_influencers = [0, 100, 250, 600] # Example breakpoints
    slopes_influencers = [10, 16, 12] # Example slopes
    @constraint(model, sum(influencers_segments[i] for i in 1:3) <= breakpoints_influencers[end])
    for i in 1:3
        @constraint(model, influencers_segments[i] <= breakpoints_influencers[i+1] - breakpoints_influencers[i])
    end


    # Weights for views and buys in the objective function
    weight_views = 0.3  # Adjust as needed
    weight_buys = 0.7  # Adjust as needed


    # BUYS
    # Example for Targeted Online Advertising
    @variable(model, targeted_s[1:3] >= 0)
    breakpoints_t = [0, 40, 310, 990] # Example breakpoints
    slopes_t = [3, 4, 5] # Example slopes
    @constraint(model, sum(targeted_s[i] for i in 1:3) <= breakpoints_t[end])
    for i in 1:3
        @constraint(model, targeted_s[i] <= breakpoints_t[i+1] - breakpoints_t[i])
    end
    
    # Example for Email Marketing
    @variable(model, email_s[1:3] >= 0)
    breakpoints_e = [0, 50, 200, 500] # Example breakpoints
    slopes_e = [2, 3, 4] # Example slopes
    @constraint(model, sum(email_s[i] for i in 1:3) <= breakpoints_e[end])
    for i in 1:3
        @constraint(model, email_s[i] <= breakpoints_e[i+1] - breakpoints_e[i])
    end
    
    # Example for Print Media
    @variable(model, print_s[1:3] >= 0)
    breakpoints_p = [0, 100, 300, 700] # Example breakpoints
    slopes_p = [1, 2, 3] # Example slopes
    @constraint(model, sum(print_s[i] for i in 1:3) <= breakpoints_p[end])
    for i in 1:3
        @constraint(model, print_s[i] <= breakpoints_p[i+1] - breakpoints_p[i])
    end
    
    # Example for Influencer Marketing
    @variable(model, influencers_s[1:3] >= 0)
    breakpoints_i = [0, 100, 250, 600] # Example breakpoints
    slopes_i = [5, 6, 7] # Example slopes
    @constraint(model, sum(influencers_s[i] for i in 1:3) <= breakpoints_i[end])
    for i in 1:3
        @constraint(model, influencers_s[i] <= breakpoints_i[i+1] - breakpoints_i[i])
    end


    @constraint(model, investment_targeted == sum(targeted_segments) + sum(targeted_s))
    @constraint(model, investment_email == sum(email_segments) + sum(email_s))
    @constraint(model, investment_print == sum(print_segments) + sum(print_s))
    @constraint(model, investment_influencers == sum(influencers_segments) + sum(influencers_s))

    expression_for_views = sum(slopes_targeted[i] * targeted_segments[i] for i in eachindex(slopes_targeted)) + 
                      sum(slopes_email[i] * email_segments[i] for i in eachindex(slopes_email)) +
                      sum(slopes_print[i] * print_segments[i] for i in eachindex(slopes_print)) +
                      sum(slopes_influencers[i] * influencers_segments[i] for i in eachindex(slopes_influencers))
    
    expression_for_buys = sum(slopes_t[i] * targeted_s[i] for i in eachindex(slopes_t)) + 
                  sum(slopes_e[i] * email_s[i] for i in eachindex(slopes_e)) +
                  sum(slopes_p[i] * print_s[i] for i in eachindex(slopes_p)) +
                  sum(slopes_i[i] * influencers_s[i] for i in eachindex(slopes_i))


    @objective(model, Max, 
        1 * (data_targeted[!, 2]*investment_targeted + data_email[!, 2]*investment_email + 
                        data_print[!, 2]*investment_print + data_influencer[!, 2]*investment_influencers) +
        1 * (data_targeted[!, 3]*investment_targeted + data_email[!, 3]*investment_email + 
                        data_print[!, 3]*investment_print + data_influencer[!, 3]*investment_influencers))

  
    
    
    # # Combined Objective Function: Maximize weighted sum of views and buys
    # @objective(model, Max, 
    #     weight_views * (expression_for_views) +  # Replace with the expression for views
    #     weight_buys * (expression_for_buys)      # Replace with the expression for buys
    # )
    
    # Budget constraint
    @constraint(model, investment_targeted + investment_email +
                       investment_print + investment_influencers <= Budget)
    
    # Solve the model
    optimize!(model)
    
    # Results
    println("Optimal Investments with Budget = ", Budget)
    println("Targeted Online Advertising: \$", value(investment_targeted))
    println("Email Marketing: \$", value(investment_email))
    println("Print Media: \$", value(investment_print))
    println("Influencer Marketing: \$", value(investment_influencers))
    println("Total Views: ", value(expression_for_views))
    println("Total Buys: ", value(expression_for_buys))

end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-31
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37 rows, 28 columns and 80 nonzeros
Model fingerprint: 0xb0153ca0
Variable types: 28 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 100 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a 